<a href="https://colab.research.google.com/github/MinhLuong0968/WebCK/blob/main/YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
mkdir AIChallengeYolov5

In [6]:
cd /content/drive/MyDrive/AIChallengeYolov5

/content/drive/MyDrive/AIChallengeYolov5


In [ ]:
ls

drive/  sample_data/


In [ ]:
# clone YOLOv5 and reset to a specific git checkpoint that has been verified working
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 68211f72c99915a15855f7b99bf5d93f5631330f

Cloning into 'yolov5'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 3520 (delta 14), reused 15 (delta 6), pack-reused 3489
Receiving objects: 100% (3520/3520), 6.82 MiB | 9.98 MiB/s, done.
Resolving deltas: 100% (2348/2348), done.
/content/drive/.shortcut-targets-by-id/1bIpUQHDDHjj1Zt1f-7jF4jki0hiGxfnI/AIChallengeYolov5/yolov5
HEAD is now at 68211f7 FROM nvcr.io/nvidia/pytorch:20.10-py3 (#1553)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 276kB 20.7MB/s 
Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)


# Download Correctly Formatted Custom Dataset 

We'll download our dataset from Roboflow. Use the "**YOLOv5 PyTorch**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.

To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).



![YOLOv5 PyTorch export](https://i.imgur.com/5vr9G2u.png)


In [ ]:
%cd /content/drive/MyDrive/AIChallengeYolov5/yolov5/data
!gdown --id 1onsbmBLR279EG1Sgx7YfQ8LsKgBoKDQn -O train.zip
!unzip -j -qq train.zip -d train

/content/drive/MyDrive/AIChallengeYolov5/yolov5/data
Downloading...
From: https://drive.google.com/uc?id=1onsbmBLR279EG1Sgx7YfQ8LsKgBoKDQn
To: /content/drive/MyDrive/AIChallengeYolov5/yolov5/data/train.zip
1.89GB [00:15, 119MB/s] 


In [ ]:
!unzip -j -qq valid.zip -d valid

In [ ]:
!unzip -j -qq train.zip -d train

In [ ]:
import os
from PIL import Image

DATASET_PATH = '/content/drive/MyDrive/AIChallengeYolov5/yolov5/train/'
# Đọc tất cả các tệp tin trong thư mục dữ liệu train
images_list = os.listdir(DATASET_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm file hình)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']
# Thêm đường dẫn DATASET_PATH vào trước tên tệp tin
images_list = [os.path.join(DATASET_PATH, filename) for filename in images_list]

for filename in images_list:
    
    # Đọc ảnh để lấy kích thước của ảnh
    img = Image.open(filename)
    width, height = img.size

    # Tìm tệp tin chứa nhãn dữ liệu tương ứng với ảnh
    annotation_file = os.path.splitext(filename)[0] + '.txt'

    # Đọc tệp tin chứa nhãn dữ liệu và tách thành từng dòng
    with open(annotation_file, 'r') as fi:
        lines = fi.read().splitlines()

    # Mở lại tệp tin nhãn để ghi đè
    with open(annotation_file, 'w') as fo:
        # Duyệt qua từng dòng trong tệp tin
        for line in lines:
            # Tách nội dung của từng dòng theo định dạng của BTC
            line = line.strip().split()
            # Chuyển đổi nội dung từ string sang int
            line = [int(x) for x in line]

            # Tiền xử lý: nếu các tọa độ ko nằm trong ảnh thì xóa logo đi (đây là nhiễu)
            if line[1] <= 0 or  line[3] > width:
                continue
            if line[2] <= 0 or  line[4] > height:
                continue

            # Chuyển tọa độ xmin, ymin, xmax, ymax về tọa độ x_center, y_center, w, h
            # Lưu ý: YOLOv3 yêu cầu chuẩn hóa các tọa độ về khoảng [0; 1].
            line = [
                line[0],
                (line[3] + line[1])/2/width,    # x_center = (xmax + xmin)/2
                (line[4] + line[2])/2/height,   # y_center = (ymax + ymin)/2
                (line[3] - line[1])/width,      # w = (xmax - xmin)
                (line[4] - line[2])/height,     # h = (ymax - ymin)
            ]

            # Tiền xử lý: nếu w hoặc h của logo bằng 0 thì bỏ qua logo này
            if line[3] <= 0 or line[4] <= 0:
                continue

            # Tiền xử lý: nếu tỉ lệ giữa w và h lớn hơn 10 thì bỏ qua logo này
            # (logo thường là hình vuông hoặc hình chữ nhật với tỉ lệ cân đối)
            if line[3] / line[4] > 10 or line[4] / line[3] > 10:
                continue

            # Chuyển các tọa độ về kiểu string
            line = [str(x) for x in line]
            # Đưa về định dạng của YOLOv
            line = ' '.join(line) + '\n'
            # Ghi vào tệp tin output
            fo.write(line)

In [ ]:
ls

data/       hubconf.py  README.md         train/          utils/
detect.py   LICENSE     requirements.txt  train.py        valid/
Dockerfile  models/     test.py           tutorial.ipynb  weights/


In [ ]:
cd /content/drive/MyDrive/AIChallengeYolov5/yolov5/data

/content/drive/.shortcut-targets-by-id/1bIpUQHDDHjj1Zt1f-7jF4jki0hiGxfnI/AIChallengeYolov5/yolov5/data


In [ ]:
import numpy as np
import pandas as pd
import shutil
import tqdm.auto as tqdm 
import os

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat data.yaml



# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: /content/drive/MyDrive/AIChallengeYolov5/yolov5/train 
val: /content/drive/MyDrive/AIChallengeYolov5/yolov5/valid 


# number of classes
nc: 6

# class names
names: ['Logo UIT','Logo HSV','Logo khoa CS','Logo khoa CE','Logo khoa SE','Logo khoa ISE']


# Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

You do not need to edit these cells, but you may.

In [ ]:
# define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /content/drive/MyDrive/AIChallengeYolov5/yolov5/models/yolov5s.yaml


# parameters
nc: 6  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))


In [ ]:
%%writetemplate /content/drive/MyDrive/AIChallengeYolov5/yolov5/models/yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=144b038ba6b747e9db04bd83b8df084d06b27e474bb55988190e1d4ae1f73c35
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 250.3 MB
GPU RAM Free: 15069MB | Used: 10MB | Util   0% | Total 15079MB


# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/drive/MyDrive/AIChallengeYolov5/yolov5/
!python train.py --img 416 --batch 64 --epochs 2000 --data './data/data.yaml' --cfg /content/drive/MyDrive/AIChallengeYolov5/yolov5/models/yolov5s.yaml --weights '' --name yolov5s_results  --cache

### Curious? Visualize Our Training Data with Labels

After training starts, view `train*.jpg` images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/drive/MyDrive/AIChallengeYolov5/yolov5/
!python test.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source /content/drive/MyDrive/AIChallengeYolov5/yolov5/TEST

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/drive/MyDrive/AIChallengeYolov5/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
%cd /content/drive/MyDrive/AIChallengeYolov5/yolov5/data/
!gdown --id 1Ng8UWSg2XNMWS49qhchTz24wSfE80cfA -O warmup.zip
!unzip -j -qq warmup.zip -d warmup

/content/drive/.shortcut-targets-by-id/1bIpUQHDDHjj1Zt1f-7jF4jki0hiGxfnI/AIChallengeYolov5/yolov5/data
Downloading...
From: https://drive.google.com/uc?id=1Ng8UWSg2XNMWS49qhchTz24wSfE80cfA
To: /content/drive/.shortcut-targets-by-id/1bIpUQHDDHjj1Zt1f-7jF4jki0hiGxfnI/AIChallengeYolov5/yolov5/data/warmup.zip
370MB [00:03, 108MB/s]


In [21]:
%cd /content/drive/MyDrive/AIChallengeYolov5/yolov5/


/content/drive/MyDrive/AIChallengeYolov5/yolov5


In [25]:
# Install the full version 7zip.
!apt-get install p7zip-full
!7z  x  -pkZ9j@mna TEST2_password.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 2918569979 bytes (2784 MiB)

Extracting archive: TEST2_password.zip
 68% 4096 Open              --
Path = TEST2_password.zip
Type = zip
Physical Size = 2918569979

  0%      0% 1 - TEST2/TEST2_0.jpg                            0% 9        0% 25         0% 35 - TEST2/TEST2_1028.jpg                                0% 41 - TEST2/TEST2_10

In [ ]:
import os
import random
random.seed(42)

WARMUP_PATH = '/content/drive/MyDrive/AIChallengeYolov5/yolov5/data/TEST2_password'
# Đọc tất cả các tệp tin trong thư mục dữ liệu warmup
images_list = os.listdir(WARMUP_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm những file ảnh)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']
# Thêm đường dẫn /content/drive/MyDrive/AIChallenge/darknet/data/warmup/ vào trước tên tệp tin
images_list = [os.path.join(WARMUP_PATH, filename) for filename in images_list]

# Sắp xếp lại danh sách
images_list.sort()

# Cuối cùng là xuất danh sách này ra tệp tin txt
f = open("warmup.txt", "w")
f.write("\n".join(images_list))
f.close()

In [26]:
cd /content/drive/MyDrive/AIChallengeYolov5/yolov5

/content/drive/MyDrive/AIChallengeYolov5/yolov5


In [ ]:
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --save-txt --img 416 --conf 0.4 --source /content/drive/MyDrive/AIChallengeYolov5/yolov5/TEST2

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', exist_ok=False, img_size=416, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=True, source='/content/drive/MyDrive/AIChallengeYolov5/yolov5/TEST2', update=False, view_img=False, weights=['runs/train/yolov5s_results/weights/best.pt'])
Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)

Fusing layers... 
Model Summary: 232 layers, 7260003 parameters, 0 gradients
image 1/6000 /content/drive/MyDrive/AIChallengeYolov5/yolov5/TEST2/TEST2_0.jpg: 288x416 2 Logo UITs, Done. (0.014s)
image 2/6000 /content/drive/MyDrive/AIChallengeYolov5/yolov5/TEST2/TEST2_1.jpg: 416x288 1 Logo UITs, Done. (0.014s)
image 3/6000 /content/drive/MyDrive/AIChallengeYolov5/yolov5/TEST2/TEST2_10.jpg: 288x416 Done. (0.012s)
image 4/6000 /content/drive/MyDrive/AIChallengeYolov5/yolov5/TEST2/TEST2_100.jpg: 288x416 Done. (0.015s)
image 5/6000 /content/drive/MyDrive/AIChallengeYolov5/yolov5/TEST2/TEST2_1000.jpg

In [30]:
import os
from PIL import Image

WARMUP_PATH = '/content/drive/MyDrive/AIChallengeYolov5/yolov5/runs/detect/exp/'
# Đọc tất cả các tệp tin trong thư mục dữ liệu train
images_list = os.listdir(WARMUP_PATH)
# Lọc bỏ những tệp tin có đuôi .txt (chỉ quan tâm file hình)
images_list = [filename for filename in images_list if not filename[-4:] == '.txt']

submission_file = 'submission.txt'

with open(submission_file, 'w') as fo:
    for filename in images_list:
        
        # Thêm đường dẫn /content/drive/MyDrive/AIChallengeYolov5/yolov5/data/warmup vào trước tên tệp tin
        file_path = os.path.join(WARMUP_PATH, filename)

        # Đọc ảnh để lấy kích thước của ảnh
        img = Image.open(file_path)
        width, height = img.size

        # Tìm tệp tin chứa nhãn dữ liệu mô hình dự đoán tương ứng với ảnh
        annotation_file = os.path.splitext(file_path)[0] + '.txt'
     
        # Đọc tệp tin chứa nhãn dữ liệu mô hình dự đoán và tách thành từng dòng
        with open(annotation_file, 'r') as fi:
            lines = fi.read().splitlines()

        # Duyệt qua từng dòng trong tệp tin
        for line in lines:
            # Tách nội dung của từng dòng ra thành các tọa độ
            line = line.strip().split()
            # Chuyển đổi nội dung từ string sang float
            line = [float(x) for x in line]
            # Chuyển định dạng class x_center y_center w h về định dạng submission.
            line = [
                filename,
                int(line[0]),
                int((line[1] - line[3]/2) * width),    # x_min = x_center - w/2
                int((line[2] - line[4]/2) * height),   # y_min = y_center - h/2
                int((line[1] + line[3]/2) * width),    # x_max = x_center + w/2
                int((line[2] + line[4]/2) * height),   # y_max = y_center + h/2
            ]
            # Chuyển các tọa độ về kiểu string
            line = [str(x) for x in line]
            # Đưa về định dạng submission của BTC (cách nhau bởi dấu phẩy)
            line = ', '.join(line) + '\n'
            # Ghi vào tệp tin output
            fo.write(line)

FileNotFoundError: ignored

In [ ]:
import math, requests

def submit_prediction(df, comment='', **kwargs):
    # Token các bạn lấy từ trang submission của cuộc thi và dán vào dòng bên dưới
    TOKEN='147778ec2d5ee9d5a3a576f7bd6dd9e32c710c878d805c7acf56377cd4eaae98a18ddfc9ae214a74622d0ae0914470faea3f60ec0cc2f8837f2e277ca8b65434' 
    URL='http://submission.mmlab.uit.edu.vn/api/submissions'
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': open(df, 'rb')},data={'comment':comment})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} seconds.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0))))
    if r.status_code != 200:
        raise Exception(r.text)
        
submit_prediction('submission.txt', comment='Submitted from Colab Notebook.')

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cp /content/yolov5/runs/exp0_yolov5s_results/weights/best.pt /content/gdrive/My\ Drive

## Congrats!

Hope you enjoyed this!

--Team [Roboflow](https://roboflow.ai)